In [201]:
# Read a json from ../2024oct
filename1 = "../2024oct/ResumenGeneral_P_DPTOS.json"
filename2 = "../2024oct/ResumenGeneral_P.json"

#Parse the json
import json
with open(filename1, encoding='utf-8') as f:
    data_dptos = json.load(f)
with open(filename2, encoding='utf-8') as f:
    data = json.load(f)

In [229]:
#Define auxiliary functions
import numpy as np
def print_keys(data, indent=0):
    if isinstance(data, dict):
        for key, value in data.items():
            print("\t" * indent + key)
            if isinstance(value, list):
                for item in value:
                    print_keys(item, indent + 1)
    elif isinstance(data, list):
        for item in data:
            print_keys(item, indent)

def  dhondt(data, Nseats):
    """
    Calculate the number of seats each party gets using the D'Hondt method
    """
    #data must be a list of numbers or np.array
    votes = np.array(data)
    seats = np.zeros(len(votes), dtype=int)
    #Calculate the number of seats
    for i in range(Nseats):
        quotients = votes / (seats + 1)
        idx = np.argmax(quotients)
        seats[idx] += 1
    return seats

def my_method(data, Nseats):
    #Rules:
    #1. Give a seat to the largest party
    #2. Consider the representation quotioent of all parties with seats, compute the new quotient if a seat was given to the party with the largest quotient
    #3. If no non-seat party has a better entry quotient, assign the seat to the party with the largest quotient. Else, give it to the largest non-seat party.
    #4. Repeat until all seats are assigned
    votes = np.array(data)
    seats = np.zeros(len(votes), dtype=int)
    largest = np.argmax(votes)
    seats[largest] += 1
    for i in range(1,Nseats):
        quotients = votes / seats
        max_quotient = np.max(quotients[seats>0])
        idx_max = np.find(quotients==max_quotient)
        next_quotient = votes / (seats + 1)
        max_out_quotient = np.max(next_quotient[seats==0])
        if next_quotient[idx_max] > max_out_quotient:
            seats[idx_max] += 1
        else:
            idx = np.find(max_out_quotient==next_quotient)
            seats[idx] += 1
    return seats

def print_diputados(asignacion, lemas, deptos):
    lemas_with_nonzero = np.sum(asignacion,axis=0) > 0
    lemas = [lema for i, lema in enumerate(lemas) if sum(asignacion[:,i])>0]
    asignacion = asignacion[:,lemas_with_nonzero]
    print('Lemas:',lemas)
    for dpto in range(19):
        print('\t', deptos[dpto], str(sum(asignacion[dpto,:]))+'/'+str(preasignacion_deptos[dpto])+':', end=' ')
        for i, lema in enumerate(lemas):
            print(asignacion[dpto,i], end=', ')
        print()

In [238]:
# SENADO
datos_eleccion_nacional = data[0]['EleccionNacional']
lemas = []
votos = []
for item in datos_eleccion_nacional:
    lm = item['LemaNombre'].rstrip() #remove trailing space
    lemas.append(lm)
    votos.append(item['Total'])
print(lemas)
print(votos)
votos_lemas = votos
senado = dhondt(votos, 30)
diputados = dhondt(votos, 99)
#print(senado)
#print(diputados)

#Ahora, para cada lema, hacer la asignación de senadores entre sublemas, y en los sublemas, entre candidatos/listas
print('*****Senado*****')
for i, lema in enumerate(lemas):
    if senado[i] == 0:
            continue
    print('***',lema, ': ',senado[i])
    datos_senado = datos_eleccion_nacional[i]['Senado']['Sublemas']
    votos_sublemas = []
    sublemas = []
    for sublema in datos_senado:
        slm  = sublema['Nombre']
        sl_votos = sublema['Total']
        sublemas.append(slm)
        votos_sublemas.append(sl_votos)
    #print(sublemas)
    #print(votos_sublemas)
    senadores_sublemas = dhondt(votos_sublemas, int(senado[i]))
    #print(senadores_sublemas)
    for j, sublema in enumerate(sublemas):
        if senadores_sublemas[j] == 0:
            continue
        print('*SUBLEMA: '+ sublema + ' ', senadores_sublemas[j])
        listas = datos_senado[j]['ListasSenado']
        votos_candidatos = []
        candidatos_nombres = []
        for lista in listas:
            #print(lista)
            candidatos = lista['Descripcion']
            votos_candidato = lista['Total']
            candidatos_nombres.append(candidatos)
            votos_candidatos.append(votos_candidato)
        
        #print(votos_candidatos)
        senadores_candidatos = dhondt(votos_candidatos, int(senadores_sublemas[j]))
        #Imprimir resultados para candidatos que obtuvieron al menos un senador
        for idx in range(len(senadores_candidatos)):
            if senadores_candidatos[idx] > 0:
                print(candidatos_nombres[idx], senadores_candidatos[idx])


['Partido Frente Amplio', 'Partido Nacional', 'Partido Colorado', 'Partido Cabildo Abierto', 'Partido Independiente', 'Partido Ecologista Radical Intransigente', 'Partido Asamblea Popular', 'Partido Identidad Soberana', 'Partido Por Los Cambios Necesarios (PCN)', 'Partido Constitucional Ambientalista', 'Partido Avanzar Republicano']
[1057515, 644147, 385685, 59000, 41206, 9152, 9951, 64735, 3128, 11691, 1866]
*****Senado*****
*** Partido Frente Amplio :  16
*SUBLEMA: Por un Uruguay para la gente  12
CARBALLO da COSTA, Felipe  - GONZALVEZ ZAS, Aida Cristina - CABRERA ESPOSITO, Fernando Javier 1
SANCHEZ PEREIRA, Alejandro Dario - RODRIGUEZ GONZALEZ, Blanca Isabel - CARRERA LEAL, Charles Daniel 9
BERGARA DUQUE, Mario Esteban - KECHICHIAN GARCIA, Norma Liliam - MAHIA FREIRE, José Carlos 1
LUSTEMBERG HARO, Maria Cristina - ORTUÑO SILVA, Edgardo Ulises - ZAVALA RABAJOLI, Alejandro  1
*SUBLEMA: SOCIALISMO ES LIBERTAD  1
CIVILA LOPEZ, Gonzalo Martin - XAVIER YELPO, Antonieta Monica - NUNES GER

In [233]:
#DIPUTADOS
#La asignación de diputados es particularmente compleja en Uruguay, ya que se asignan por departamento
#La corte electoral hace una primera asignación de bancas por departamento que se utilizan para el procedimiento siguiente.
#La primera asignación es por lema, y esto se hace en forma proporcional a los votos obtenidos a nivel nacional.  Esto nunca pueda cambiar.
#Luego, en cada departamento, se asignan diputados que se pagan 'enteros' en cada lema
#Luego, se asignan hasta completar dos diputados por departamento, en forma proporcional a los votos obtenidos por cada lema en el departamento
#Luego, se genera una tabla de cocientes (dhondt) combinando todos los lemas y departamentos, y se van asignando en forma decreciente
deptos = ['Montevideo','Canelones','Maldonado','Rocha','Treinta y Tres','Cerro Largo','Rivera', 
          'Artigas','Salto','Paysandú', 'Río Negro', 'Soriano', 'Colonia','San José', 'Flores', 
          'Florida','Durazno','Lavalleja','Tacuarembó']
preasignacion_deptos = np.array([38,16,5,2,2,2,3,2,4,3,2,2,4,3,2,2,2,2,3])
votos_validos = data[0]['TotalVotosALemas']
costo_banca_entera = votos_validos / 99
print('Costo entero: ', costo_banca_entera)

# Crear matriz con datos de votos por lema, por departamento
votos_lemas_dptos = np.zeros((19, len(lemas)), dtype=int)
diputados_lemas_dptos = np.zeros((19, len(lemas)), dtype=int)
for datos_depto in data_dptos:
    nombre_dpto = datos_depto['DepNombre']
    # find nombre_dpto in deptos, get idx
    preasig_idx = deptos.index(nombre_dpto)
    preasig = preasignacion_deptos[preasig_idx]
    #print(nombre_dpto,', bancas: ', preasig)
    datos_votacion = datos_depto['EleccionNacional']
    for i, lema in enumerate(datos_votacion):
        votos_lema_dpto = lema['Total']
        #print(lema['LemaNombre'].rstrip(), lema['Total'])
        votos_lemas_dptos[preasig_idx,i] = votos_lema_dpto
        diputados_lemas_dptos[preasig_idx,i] = np.floor(votos_lema_dpto / costo_banca_entera)
#Check: sum of votos_lemas_dptos must match votos_lemas
assert all(votos_lemas_dptos.sum(axis=0) == votos_lemas)

#Ronda 1: asignar enteros
print('*****Diputados*****')
#print('Lemas: ', lemas)
#print('Diputados por lema: ', diputados)
#print('***Ronda 1: Asignar enteros')
#print_diputados(diputados_lemas_dptos, lemas, deptos)

#Ronda 2: completar 2 por departamento
#print('***Ronda 2: Completar 2 por departamento')
for dpto in range(19):
    if sum(diputados_lemas_dptos[dpto,:]) >= 2:
        continue
    votos_dpto = votos_lemas_dptos[dpto,:]
    diputados_dpto = dhondt(votos_dpto, 2)
    diputados_lemas_dptos[dpto,:] = diputados_dpto
#print_diputados(diputados_lemas_dptos, lemas, deptos)

#Ronda 3: crear tabla de cocientes y asignar en orden decreciente
tabla_cocientes = votos_lemas_dptos / (diputados_lemas_dptos + 1)
tabla_cocientes = tabla_cocientes.flatten()
dptos_tabla_cocientes = np.repeat(deptos, len(lemas)).flatten()
lemas_tabla_cocientes = np.tile(lemas, 19).flatten()
#Ordenar tabla de cocientes y con los índices resultantes, ordenar dptos_tabla_cocientes y lemas_tabla_cocientes
indices = np.argsort(tabla_cocientes)[::-1]
tabla_cocientes = tabla_cocientes[indices]
dptos_tabla_cocientes = dptos_tabla_cocientes[indices]
lemas_tabla_cocientes = lemas_tabla_cocientes[indices]
for i,quot in enumerate(tabla_cocientes):
    if np.sum(diputados_lemas_dptos) == 99:
        break
    #Si hay bancas disponibles en el lema y en el depto, asignar
    dpto_idx = deptos.index(dptos_tabla_cocientes[i])
    lema_idx = lemas.index(lemas_tabla_cocientes[i])
    if np.sum(diputados_lemas_dptos[dpto_idx,:]) < preasignacion_deptos[dpto_idx] and sum(diputados_lemas_dptos[:,lema_idx]) < diputados[lema_idx]:
        diputados_lemas_dptos[dpto_idx,lema_idx] += 1
        #print(dptos_tabla_cocientes[i], lemas_tabla_cocientes[i], quot)
    #elif np.sum(diputados_lemas_dptos[dpto_idx,:]) >= preasignacion_deptos[dpto_idx]:
        #print(dptos_tabla_cocientes[i], lemas_tabla_cocientes[i], quot)
        #print('No asignado: departamento completo')
    #else:
        #print(dptos_tabla_cocientes[i], lemas_tabla_cocientes[i], quot)
        #print('No asignado: lema completo')

#print_diputados(diputados_lemas_dptos, lemas, deptos)
assert np.sum(diputados_lemas_dptos) == 99
assert np.all(np.sum(diputados_lemas_dptos, axis=1) == preasignacion_deptos)
assert np.all(np.sum(diputados_lemas_dptos, axis=0) == diputados)
#print('Total diputados asignados: ', np.sum(diputados_lemas_dptos))
#print('Total diputados asignados por lema: ', np.sum(diputados_lemas_dptos, axis=0))
#print('Total diputados asignados por departamento: ', np.sum(diputados_lemas_dptos, axis=1))

#Ronda 4: traslado de bancas
tabla_cocientes_final = votos_lemas_dptos / diputados_lemas_dptos
for lema in lemas:
    traslado_bancas = any(diputados_lemas_dptos[:,lemas.index(lema)] == 0) #Si no hay deptos sin bancas, no hay nada trasladable
    while traslado_bancas:
        diputados_lema = diputados_lemas_dptos[:,lemas.index(lema)]
        diputados_dptos = np.sum(diputados_lemas_dptos, axis=1)
        #Calcular cocientes asociados
        cocientes_pagos = tabla_cocientes_final[:,lemas.index(lema)]
        cociente_a_pagar = votos_lemas_dptos[:,lemas.index(lema)] / (diputados_lemas_dptos[:,lemas.index(lema)] + 1)
        #Obtener deptos sin bancas para el lema
        deptos_sin_lema = np.where(diputados_lemas_dptos[:,lemas.index(lema)] == 0)
        #Obtener el depto con mayor cociente y sin banca
        mayor_cociente_sin_banca = np.max(cociente_a_pagar[diputados_lema == 0]) #Solo trasladar si no hay bancas para este lema
        depto_mayor_cociente_sin_banca = np.where(cociente_a_pagar == mayor_cociente_sin_banca)[0][0]
        aux = cocientes_pagos[(diputados_lema > 0)*(diputados_dptos > 2)]
        if len(aux) == 0:
            traslado_bancas = False
            break
        menor_cociente_con_banca = np.min(aux) #Solo trasladar si hay al menos 3 bancas en el depto
        depto_menor_cociente_con_banca = np.where(cocientes_pagos == menor_cociente_con_banca)[0][0]
        #Si el menor cociente es menor que el mayor cociente de los deptos con bancas, trasladar la banca
        if menor_cociente_con_banca < mayor_cociente_sin_banca:
            #Trasladar banca!
            diputados_lemas_dptos[depto_mayor_cociente_sin_banca,lemas.index(lema)] += 1
            diputados_lemas_dptos[depto_menor_cociente_con_banca,lemas.index(lema)] -= 1
            print(lema, 'Trasladó banca de', deptos[depto_menor_cociente_con_banca], '(cociente:', menor_cociente_con_banca,') a', deptos[depto_mayor_cociente_sin_banca], '(cociente:', mayor_cociente_sin_banca,')')
        else:
            traslado_bancas = False

print_diputados(diputados_lemas_dptos, lemas, deptos)
assert np.sum(diputados_lemas_dptos) == 99
#assert np.all(np.sum(diputados_lemas_dptos, axis=1) == preasignacion_deptos) #Esto no es requisito luego del traslado
assert np.all(np.sum(diputados_lemas_dptos, axis=0) == diputados)
print('Total diputados asignados: ', np.sum(diputados_lemas_dptos))
print('Total diputados asignados por lema: ', np.sum(diputados_lemas_dptos, axis=0))
print('Total diputados asignados por departamento: ', np.sum(diputados_lemas_dptos, axis=1))

Costo entero:  23111.878787878788
*****Diputados*****
Partido Cabildo Abierto Trasladó banca de Salto (cociente: 3849.0 ) a Canelones (cociente: 7697.0 )
Partido Identidad Soberana Trasladó banca de Colonia (cociente: 2024.0 ) a Canelones (cociente: 10379.0 )
Lemas: ['Partido Frente Amplio', 'Partido Nacional', 'Partido Colorado', 'Partido Cabildo Abierto', 'Partido Independiente', 'Partido Identidad Soberana']
	 Montevideo 38/38: 21, 8, 6, 1, 1, 1, 
	 Canelones 18/16: 9, 4, 3, 1, 0, 1, 
	 Maldonado 5/5: 2, 2, 1, 0, 0, 0, 
	 Rocha 2/2: 1, 1, 0, 0, 0, 0, 
	 Treinta y Tres 2/2: 1, 1, 0, 0, 0, 0, 
	 Cerro Largo 2/2: 1, 1, 0, 0, 0, 0, 
	 Rivera 3/3: 1, 0, 2, 0, 0, 0, 
	 Artigas 2/2: 1, 1, 0, 0, 0, 0, 
	 Salto 3/4: 1, 1, 1, 0, 0, 0, 
	 Paysandú 3/3: 1, 1, 1, 0, 0, 0, 
	 Río Negro 2/2: 1, 1, 0, 0, 0, 0, 
	 Soriano 2/2: 1, 1, 0, 0, 0, 0, 
	 Colonia 3/4: 1, 1, 1, 0, 0, 0, 
	 San José 3/3: 1, 1, 1, 0, 0, 0, 
	 Flores 2/2: 1, 1, 0, 0, 0, 0, 
	 Florida 2/2: 1, 1, 0, 0, 0, 0, 
	 Durazno 2/2: 1, 1,

C:\Users\pablo\AppData\Local\Temp\ipykernel_58748\3690299169.py:86: RuntimeWarning: divide by zero encountered in divide
  tabla_cocientes_final = votos_lemas_dptos / diputados_lemas_dptos


In [205]:
#Ahora, con la lista de diputados asignados por lema y departamento, asignar los diputados a los candidatos (listas) e imprimir
diputados_nac_acumulados_por_numero_de_lista = [[]]
numeros_nac_de_lista = [[]]
#Generar array de arrays para guardar diputados por número dpto, lema y número de lista, con tamaño 19 x len(lemas) y conteniendo listas de tamaño variable
diputados_por_numero_de_lista = np.empty((19,len(lemas)), dtype=object)
numeros_de_lista = np.empty((19,len(lemas)), dtype=object)
for data_dpto in data_dptos:
    dpto = data_dpto['DepNombre']
    i = deptos.index(dpto)
    print('\n\t*****', dpto,':', sum(diputados_lemas_dptos[i,:]) ,'/', preasignacion_deptos[i],'*****')
    for j, lema in enumerate(lemas):
        diputados_por_numero_de_lista[i,j] = []
        numeros_de_lista[i,j] = []
        if len(diputados_nac_acumulados_por_numero_de_lista) <= j:
            diputados_nac_acumulados_por_numero_de_lista.append([])
            numeros_nac_de_lista.append([])
        if diputados_lemas_dptos[i,j] == 0:
            continue
        print('\n', lema, '(', diputados_lemas_dptos[i,j],')')
        print('\t', end='')
        datos_dpto = data_dpto['EleccionNacional'][j]
        #print(datos_dpto)
        listas = datos_dpto['Hojas']
        votos_candidatos = []
        candidatos_nombres = []
        for lista in listas:
            candidatos = lista['HN']
            votos_candidato = lista['Tot']
            candidatos_nombres.append(candidatos)
            votos_candidatos.append(votos_candidato)
        diputados_candidatos = dhondt(votos_candidatos, int(diputados_lemas_dptos[i,j]))
        for idx in range(len(diputados_candidatos)):
            if diputados_candidatos[idx] > 0:
                #Imprimir sin insertar líneas nuevas
                print(candidatos_nombres[idx],':', diputados_candidatos[idx], end=', ')
                diputados_por_numero_de_lista[i,j].append(diputados_candidatos[idx])
                numeros_de_lista[i,j].append(candidatos_nombres[idx])
                #Acumular diputados por número de lista
                if candidatos_nombres[idx] in numeros_nac_de_lista[j]:
                    idx_lista = numeros_nac_de_lista[j].index(candidatos_nombres[idx])
                    diputados_nac_acumulados_por_numero_de_lista[j][idx_lista] += diputados_candidatos[idx]
                else:
                    numeros_nac_de_lista[j].append(candidatos_nombres[idx])
                    diputados_nac_acumulados_por_numero_de_lista[j].append(diputados_candidatos[idx])
                #Guardar diputados por número de lista por depto por lema
                

    print('\n')

#Imprimir diputados acumulados por número de lista
print('Diputados nacionales acumulados por número de lista:')
#Ordenar por diputados: primero los que tienen más diputados
for i in range(len(diputados_nac_acumulados_por_numero_de_lista)):
    dips_acum = diputados_nac_acumulados_por_numero_de_lista[i]
    nums = numeros_nac_de_lista[i]
    if len(dips_acum) == 0:
        continue
    dips_acum,nums = zip(*sorted(zip(dips_acum, nums), reverse=True))
    print(lemas[i])
    for j in range(len(dips_acum)):
        print(nums[j],':',dips_acum[j], end=', ')
    print('')


	***** Artigas : 2 / 2 *****

 Partido Frente Amplio ( 1 )
	609 : 1, 
 Partido Nacional ( 1 )
	2525 : 1, 


	***** Canelones : 18 / 16 *****

 Partido Frente Amplio ( 9 )
	609 : 7, 95939 : 1, 1001993311 : 1, 
 Partido Nacional ( 4 )
	400 : 2, 3340 : 2, 
 Partido Colorado ( 3 )
	10 : 2, 25 : 1, 
 Partido Cabildo Abierto ( 1 )
	510 : 1, 
 Partido Identidad Soberana ( 1 )
	18010 : 1, 


	***** Cerro Largo : 2 / 2 *****

 Partido Frente Amplio ( 1 )
	609 : 1, 
 Partido Nacional ( 1 )
	3 : 1, 


	***** Colonia : 3 / 4 *****

 Partido Frente Amplio ( 1 )
	95609 : 1, 
 Partido Nacional ( 1 )
	904 : 1, 
 Partido Colorado ( 1 )
	10 : 1, 


	***** Durazno : 2 / 2 *****

 Partido Frente Amplio ( 1 )
	1001971 : 1, 
 Partido Nacional ( 1 )
	40 : 1, 


	***** Flores : 2 / 2 *****

 Partido Frente Amplio ( 1 )
	95609 : 1, 
 Partido Nacional ( 1 )
	40 : 1, 


	***** Florida : 2 / 2 *****

 Partido Frente Amplio ( 1 )
	959609 : 1, 
 Partido Nacional ( 1 )
	62 : 1, 


	***** Lavalleja : 2 / 2 *****

 P

In [237]:
#Ahora, hacer una ingenería inversa para poner nombres a las listas que sacaron diputados
for datos_dpto in data_dptos:
    dpto = datos_dpto['DepNombre']
    i = deptos.index(dpto)
    print('\n', dpto, '(' +str(sum(diputados_lemas_dptos[i,:]))+')', end='')
    for j , lema in enumerate(lemas):
        if diputados_lemas_dptos[i,j] == 0:
            continue
        print('\n\t', lema, '('+str(diputados_lemas_dptos[i,j])+')')
        
        if diputados_lemas_dptos[i,lemas.index(lema)] == 0:
            continue
        #Recorrer listas con diputados, buscar número de votos, con el número de votos, ir a datos_dpto['Diputdos'] y buscar el item cuyo 'VotosHojas' coincida, retrieve Descripcioon
        for idx in range(len(diputados_por_numero_de_lista[i,lemas.index(lema)])):
            dips = diputados_por_numero_de_lista[i,lemas.index(lema)][idx]
            num = numeros_de_lista[i,lemas.index(lema)][idx]
           #print(numero, diputados)
            for lista in datos_dpto['EleccionNacional'][lemas.index(lema)]['Hojas']:
                #print(lista['HN'], lista['Tot'])
                if lista['HN'] == num:
                    votos = lista['Tot']
                    #print(votos)
                    break
            for lista in datos_dpto['EleccionNacional'][lemas.index(lema)]['Diputados']['Listas']:
                #print(lista)
                if lista['VotosHojas'] == votos:
                    print('\t\t'+str(num)+'('+str(dips)+'):', lista['Descripcion'])
                    break
        


 Artigas (2)
	 Partido Frente Amplio (1)
		609(1): LORENZO MACHADO, Eduardo Nicolas - GUICHON ETCHEVERRY, Nerysabel 

	 Partido Nacional (1)
		2525(1): SORAVILLA PINATO, Emiliano . - VILLAMOR JIMENEZ, Fernanda 

 Canelones (18)
	 Partido Frente Amplio (9)
		609(7): IRIGOIN MACARI, Pedro Augusto - ETCHEVERRY LIMA, Lucia Ines - PREVE COCCO, Federico Mariano
		95939(1): MAHIA FREIRE, José Carlos - SAFFORES VARELA, Sandra Rosemary - RODRIGUEZ MARENCO, Javier Rubi
		1001993311(1): DIVERIO VIERA, Daniel Israel - MUÑIZ JIMENEZ, Maria Susana - DUARTE SUAREZ, Edgardo 

	 Partido Nacional (4)
		400(2): NIFFOURI BLANCO, Amin  - DELGADO MAS, Juan Pablo - IMODA PENELA, Rosa Maria
		3340(2): ANDUJAR ALVAREZ DE RON, Sebastian  - DASTUGUE da ROSA, Alvaro  - VAZQUEZ CEDRÉS, Mariela Dorinel

	 Partido Colorado (3)
		10(2): CERVINI PRATTO, Walter Santiago - de ARMAS GONZÁLEZ, María Paula - ALVEAR GONZALEZ, Jorge Jose
		25(1): DUQUE BARRETO, Matías Heber - CASTRO BRUNIG, Diego Alberto - BURGUEÑO MORENO, 